Custom Resrouces (CRs) are born from Custom Resource Definitions (CRDs) which are themselves native Kubernetes types. These resources allow you to make your own config data for objects or applications. Operators are ALWAYS paired with CRs (that is what sets an operator apart from a controller – the fact that it spawns a CR for its own use or surveilance); controllers can look after CRs but they won't spin them up directly. You don't want to be writing status fields manually and you don't want controllers writing spec fields automatically so a *spec-status split* is usually employed. Since the *status/* endpoint is itself a seperate endpoint, Roles are usually used to limit the access of controllers to what they can and cannot update about a given object / resource. The client provided by client-go does not know about your CRs by default. There are 3 options this book provides: 
- Dynamic Client: Least type safety and least amount of completeness features provided by the Kube API. This client does not know ANY Go types besides *unstructured.Unstructred*! This client just takes the object at the endpoint and marshals it into JSON. *You can update stuff this way but it's all through manipulating JSON!!* Since this is very generic, generic controllers like garbage collectors are the ones that use this client (this client uses a *map[string]interface{}* which is how its so generic)
- Typed Clients: These clients use actual Go types! The tradeoff here is that you get increased type safety and better code but at the cost of flexibility, having to know all your types at compile time, and it adds complexity. Since these are actual types, it's good to have your custom types in a seperate Go module with the structure like so: *pkg/apis/group/version* && define your Kind's Go struct in *types.go*. Also remember that every Kind needs *TypeMeta* and that if your CR has its own endpoint you also need to implement its *ObjectMeta*. Along with your *types.go* file there are others we have to consider! The *doc.go* file describes the APIs purpose and some generation tags. *register.go* helps register the Go struct into the scheme i.e. translate the Go struct into a GVR which can then be translated into a GVK and then an actual HTTP endpoint! **Look at page 118 for the good stuff on how schemes are used!** *zz_generated.deepcopy.go* defines the deep copy methods for your struct!
    - Typed Client via client-gen: client-gen will generate code for you and basically glue together stuff you've implemented – those interfaces and structs for your object – and create a clientset for your Kind just like anyother --> this is done using the **NewForConfig** helper function. 
    - I didn't see a second one?
- controller-runtime Client of Operator SDK and Kubebuilder. This client is somewhat of a mix between the two previous ones. Whereas the dynamic client can handle any object in Kubernetes without having to know about it runtime and Typed Clients need to have everything ready at compile time and then produce a client for each resource and its namespace, the controller-runtime client can deal with ANY object so-long as it is registerd into the Scheme beforehand! With your own custom types you'll have to import your scheme so that the client knows what you're trying to ask for. The example below showcases the difference between custom schemes and default schemes for controller runtime.

In [ ]:
// Scheme with no custom resources

import (
    "flag"

    corev1 "k8s.io/api/core/v1"
    metav1 "k8s.io/apimachinery/pkg/apis/meta/v1"
    "k8s.io/client-go/kubernetes/scheme"
    "k8s.io/client-go/tools/clientcmd"

    runtimeclient "sigs.k8s.io/controller-runtime/pkg/client"
)

kubeconfig = flag.String("kubeconfig", "~/.kube/config", "kubeconfig file path")
flag.Parse()
config, err := clientcmd.BuildConfigFromFlags("", *kubeconfig)
cl, _ := runtimeclient.New(config, client.Options{
    Scheme: scheme.Scheme,
})
podList := &corev1.PodList{}
err := cl.List(context.TODO(), client.InNamespace("default"), podList)

// Scheme with custom resources

import (
    "flag"

    corev1 "k8s.io/api/core/v1"
    metav1 "k8s.io/apimachinery/pkg/apis/meta/v1"
    "k8s.io/client-go/kubernetes/scheme"
    "k8s.io/client-go/tools/clientcmd"

    runtimeclient "sigs.k8s.io/controller-runtime/pkg/client"
    cnatv1alpha1 "github.com/.../cnat/cnat-kubebuilder/pkg/apis/cnat/v1alpha1" // this is the custom scheme provided in the book but it will be your own
)

kubeconfig = flag.String("kubeconfig", "~/.kube/config", "kubeconfig file")
flag.Parse()

config, err := clientcmd.BuildConfigFromFlags("", *kubeconfig)

crScheme := runtime.NewScheme() // make a custom scheme from the existing one
cnatv1alpha1.AddToScheme(crScheme) // add your structs to the current scheme so your type can be translated into a GVK

cl, _ := runtimeclient.New(config, client.Options{
    Scheme: crScheme,
})
list := &cnatv1alpha1.AtList{}
err := cl.List(context.TODO(), client.InNamespace("default"), list)

This controller-runtime client is very handy when it comes to dealing with a lot of types. Instead of your controller or operator needing many client sets for all of the objects it needs to access, there is a central client it can use for all of them. 